In [51]:
ai_model = 'gemini-2.0-flash-001'
git_repo_name = "piotrhyzy/cloudfest2025"
git_pr_number = 5

In [52]:
import configuration as config

In [53]:
import vertexai
vertexai.init(project=config.VERTEX_PROJECT_ID, location=config.VERTEX_LOCATION)

In [54]:
from shared.gcp import Secret
secret = Secret.access_secret_version(
        config.GITHUB_SECRET_PROJECT_ID,
        config.GITHUB_SECRET_NAME,
    )
app_id = secret["app_id"]
installation_id = secret["installation_id"]
private_key = secret["private_key"]

In [55]:
from shared.git import get_client
git_client = get_client(app_id, installation_id, private_key, config.GITHUB_BASE_URL)

## Fetch PR Title and Description

In [ ]:
from shared.git import get_pr_details
pr_details = get_pr_details(git_client, git_repo_name, git_pr_number)

print(pr_details.model_dump_json(indent=2))

## Fetch PR Diff

In [ ]:
from shared.git import get_pr_diff
pr_diff = get_pr_diff(git_client, git_repo_name, git_pr_number)

print(pr_diff)


## Request LLM For Summary

In [59]:
from vertexai.generative_models import GenerativeModel, GenerationConfig


def summarize_pull_request(pr_title: str, pr_description: str, pr_diff: str) -> str:
    """
    Calls the Gemini model on Vertex AI to generate a concise summary of a pull request
    based on its title, description, and code diff.
    Returns the generated summary as a string.
    """

    # prepare configuration for the model
    generation_config = {
        "max_output_tokens": 8192, # limit the length of the response
        "temperature": 0.5, # lower temperature for more deterministic output
        "top_p": 1, # more deterministic output
        # "top_k": 30, # more deterministic output
    }
    # Load the Gemini model
    model = GenerativeModel(ai_model)

    # Build the prompt in English (basic instructions, no explicit "role")
    prompt_text = f"""
        Below is a pull request:

        Title:
        {pr_title}

        Description:
        {pr_description}

        Code Diff:
        {pr_diff}

        Task:
        1. Read the pull request details and code diff.
        2. Provide a concise summary of what this PR changes and why it matters with **Highlights**.
        3. Keep it short, focusing on the main modifications and purpose.
        4. Provide Changeog using expandalbe "Changelog" section for more details.

       Summary of Changes:
        """

    # Call the model with the prompt
    response = model.generate_content(
        prompt_text,
        generation_config=generation_config,
    )
    response_text = response.text.strip()

    # Strip the code fences if present
    if response_text.startswith('```json'):
        response_text = response_text[7:]
    if response_text.endswith('```'):
        response_text = response_text[:-3]
    response_text = response_text.strip()


    # Return the model's output as a string
    return response.text


In [ ]:
summary = summarize_pull_request(pr_details.title, pr_details.description, pr_diff)
print(summary)

## Publish Summary to GitHub

In [ ]:
from shared.git import post_comment
post_comment(git_client, git_repo_name, git_pr_number, summary)